# Selenium for 법제동향

### Import Files

In [1]:
import requests
import lxml.html
import sqlite3
from pandas.io import sql
import os
import re
import string
import pandas as pd
from tabulate import tabulate
from selenium.webdriver import Chrome
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from selenium import webdriver
from selenium.webdriver import ActionChains
import csv

## get_urls()
- gets the list of urls to detail pages

In [2]:
def get_urls():
    
    page = 1
    max_page = 0
    url_list = []
    
    response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do?A=A&searchType=all&searchPageRowCnt=50&pageIndex=1')
    root = lxml.html.fromstring(response.content)
        
    # 페이지 번호 중에서 max 페이지 가져오기    
    max_page = int(root.xpath('//*[@id="contents"]/div[4]/div[1]/a[13]/text()')[0])
    
    while(True):
        
        response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do?A=A&searchType=all&searchPageRowCnt=50&pageIndex={}'\
                               .format(page))
        root = lxml.html.fromstring(response.content)
        for tr in root.xpath('//*[@id="contents"]/table/tbody/tr') :
            a = tr.xpath('td[3]/a')[0]
            url = 'https://world.moleg.go.kr' + a.get('href')
            url_list.append(url)

        if (page >= max_page):
            break
        else:
            page = page + 1

        time.sleep(1)
        
    return url_list

## remove_punc()
- removes punctuations from string

In [3]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△《》•‘’○※▷【】]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string2 = re.sub('\n', ' ', new_string) # newline 제거
    new_string3 = re.sub('  ', ' ', new_string2) # double space 제거
    new_string4 = re.sub('   ', ' ', new_string3) # double space 제거
    return new_string4

## db_save()
- saves data as db file

In [4]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

## get_info()
- returns detailed information on the article pages as list

In [5]:
def get_info(url):
    details = []
    
    browser.get(url)
    title = browser.find_element('xpath','//*[@id="contents"]/div[3]').text
    country = browser.find_element('xpath','//*[@id="contents"]/div[2]/div[2]/strong').text
    upload_date = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[1]/span').text
    views = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[2]/span').text
    trs = browser.find_element('xpath','//*[@id="lgslTrend_CTS"]').text
    
    browser.implicitly_wait(4)
    
    words = remove_punc(trs)
    details.append(title)
    details.append(country)
    details.append(upload_date)
    details.append(views)
    details.append(words)
    
    return details

## db_save_as_csv()
- saves dataframe as csv

In [27]:
def db_save_as_csv(data):
    data.to_csv("법제동향.csv", index=False, encoding='utf-8')

### ---------------------------------------------------------------------------------------------------------------------
# Main
This is the main part which uses functions to read articles and saves the data as csv file
### ---------------------------------------------------------------------------------------------------------------------

In [7]:
from selenium.webdriver.chrome.options import Options

options = Options()

#바이너리 위치 설정
#options.binary_location= 'C:\Program Files\Google\Chrome\Application\chrome.exe'

urls = get_urls()
df_list = []

browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정

for url in urls:
    
    details = get_info(url)
    
    df = pd.DataFrame([{
        "제목": details[0],
        "국가": details[1],
        "등록일자": details[2],
        "조회수": details[3],
        "본문내용": details[4]
    }])
    
    df_list.append(df)
    
    browser.implicitly_wait(4)

ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)
db_save_as_csv(ARTICLE_LIST)

browser.close()
browser.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_11624\2067714548.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정
C:\Users\user\AppData\Local\Temp\ipykernel_11624\2067714548.py:11: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정


4042 건 저장완료..


## reads the saved csv file and displays the result as a dataframe

In [8]:
with open(r"법제동향_page1.csv", 'r', encoding='utf-8') as f:
    
    document = pd.read_csv(f)
    
    display(document)

,제목,국가,등록일자,조회수,본문내용
0,"말레이시아, 개정 고용법 시행 연기",말레이시아,2022.09.05.,243,말레이시아 입법동향 말레이시아 개정 고용법 시행 연기 9월 1일 자로 시행 예정이었...
1,"마카오, 원숭이두창을 《전염병 강제신고》 대상으로 지정",중국 마카오특별행정구,2022.08.01.,789,마카오 법제동향 마카오 원숭이두창을 전염병 강제신고 대상으로 지정 마카오 카지노...
2,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령,World,2022.07.22.,1597,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령 우리나라 형법제9조에 의하...
3,"독일, 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의",독일,2022.07.14.,1081,독일 입법동향 독일 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의 ...
4,"태국, 「2022년 형법전 개정법 (제29권)」 공포",태국,2022.07.12.,1109,태국 입법동향 태국 2022년 형법전 개정법 제29권 공포 태국 정부는 2021...
...,...,...,...,...,...
4037,中 150억달러 주식펀드 추진,중국,2005.10.02.,1297,동아일보 20050617 중국 정부가 침체 상태에 빠진 주식시장을 지원하기 위해 총...
4038,"중국, 금융시장 개방 가속도",중국,2005.10.02.,1345,한겨레 20050621 중국이 2006년 말 금융시장 개방을 앞두고 금융개혁 작업...
4039,포스코 ‘철강 고급화’로 中공략,중국,2005.10.02.,3951,1200만달러 투자 전기용강판 공장 2007년 착공 문화일보 20050620 포스코...
4040,"中하이얼, 메이택에 인수의향서 제출",중국,2005.10.02.,3632,머니투데이20050621 중국의 최대 가전사 하이얼이 사모펀드와 공동으로 미국 가전...


## sorts the data by view counts in descending order

In [26]:
df_no_detail = document.iloc[:, [True, True, True, True, False]]

sorted_df = df_no_detail.sort_values(by='조회수', ascending=False)
display(sorted_df)

,제목,국가,등록일자,조회수
221,세계 각국의 “몰카” 범죄 처벌 규정,World,2019.04.30.,23378
249,세계 각국의 아동학대 처벌규정,World,2019.01.29.,16946
194,마약 범죄에 대한 국가별 처벌,World,2019.08.28.,13216
114,세계 각국의 낙태 관련 규정,World,2020.12.21.,12594
2344,미국 정부 조직도,미국,2011.02.23.,11701
...,...,...,...,...
409,"인도네시아, 제16차 경제정책 패키지 발표",인도네시아,2017.10.13.,270
0,"말레이시아, 개정 고용법 시행 연기",말레이시아,2022.09.05.,243
411,"러시아 정부, 수입산 가구의 정부조달금지명령 조치",러시아,2017.10.11.,212
3201,미국 범죄자 DNA 데이터베이스 수록 법률 현황,미국,2010.11.15.,103
